In [1]:
import cv2
import matplotlib.pyplot as plt
import glob

In [2]:
#display the image
def display_img(img):
    #resize the image
    img = cv2.resize(img,(1200,1000))
    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#convert image to grayscales image
def grayscale_cvt(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


# SIFT algorithm

In [3]:
current_buffer = glob.glob('../panoramic_images/current_buffer/*.jpeg')


### lets run the SIFT algorithm for all the images in our current buffer and store contents in dictionary

In [4]:
feature_dict = {}
for filepath in current_buffer:
    img = cv2.imread(filepath)
    gray = grayscale_cvt(img)
    sift = cv2.SIFT_create()
    keypoints = sift.detect(gray,None)
    kp_img = cv2.drawKeypoints(gray,keypoints,img)
    name = filepath.split('\\')[-1].split('.')[0]
    feature_dict[name] = keypoints
    
    

## Feature matching

In [5]:
images = glob.glob('../panoramic_images/*.jpg')

In [6]:
f_dict = {}
for path in images:
    img = cv2.imread(path)
    gray = grayscale_cvt(img)
    sift = cv2.SIFT_create()
    kp,desc = sift.detectAndCompute(gray,None)
    name = path.split('\\')[-1].split('.')[0]
    f_dict[name] = [img,kp,desc]

In [7]:
img1 = f_dict['frame1']
img2 = f_dict['frame2']

In [8]:
#brute force matchers (k nearest neighbours)

bf = cv2.BFMatcher()

matches = bf.knnMatch(img1[2],img2[2],k=2)

#print(matches)

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
        
# cv.drawMatchesKnn expects list of lists as matches.
#img3 = cv2.drawMatchesKnn(img1[0],img1[1],img2[0],img2[1],good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#display_img(img3)

In [9]:
#FLANN matcher
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(img1[2],img2[2],k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv2.DrawMatchesFlags_DEFAULT)
img4= cv2.drawMatchesKnn(img1[0],img1[1],img2[0],img2[1],matches,None,**draw_params)

#display_img(img4)


## Panorama generator inbuilt opencv

In [14]:
image_list = [cv2.imread(path,cv2.IMREAD_GRAYSCALE) for path in images]

In [16]:
#doesen't work in large inputs, takes forever to complete
stitcher = cv2.Stitcher_create()

(status,stiched) = stitcher.stitch(image_list)
if status == 0:
    print('stitch successful')
else:
    print("[INFO] image stitching failed ({})".format(status))


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\stitching\src\exposure_compensate.cpp:564: error: (-215:Assertion failed) _image.type() == CV_8UC3 in function 'cv::detail::BlocksCompensator::apply'
